In [2]:
import pandas as pd
import plotly.express as px

In [5]:
df = pd.read_csv("../data_raw/lrf_public_file_review.tab", sep="\t", engine="python")

In [382]:
relevant_keys = {
    "L3_A": "In your own words, what is the greatest source of RISK TO YOUR SAFETY in your daily life?",
    "L3_B": "Other than what you just mentioned in your own words what is another MAJOR source of RISK TO YOUR SAFETY in your daily life?",
    "L5": "Do you think that climate change is a very serious threat, a somewhat serious threat, or not a threat at all to the people in this country in the next 20 years?"
}

In [183]:
survey_columns = df.columns[df.columns.str.startswith("L")].to_list()
drop_columns = [item for item in survey_columns if item not in list(relevant_keys.keys())]
df_relevant = df.drop(drop_columns, axis=1)

In [374]:
bins = [15, 18, 31, 65, 100]
labels = ["15 to 17", "18 to 30", "31 to 64", "65+"]

df_relevant["AgeGroup"] = pd.cut(df_relevant["Age"], bins=bins, labels=labels, right=False)

In [375]:
l3_a = df_relevant.pivot_table(index=["Country", "AgeGroup"], columns="L3_A", values="projection_weight", aggfunc='sum').astype(int)
l3_a = l3_a.rename_axis(None, axis=1)
l3_a.columns = l3_a.columns.astype(str)

l3_b = df_relevant.pivot_table(index=["Country", "AgeGroup"], columns="L3_B", values="projection_weight", aggfunc='sum').astype(int)
l3_b = l3_b.rename({" ": "99"}, axis=1)
l3_b = l3_b.rename_axis(None, axis=1)

l3 = l3_a + l3_a

l5 = df_relevant.pivot_table(index=["Country", "AgeGroup"], columns="L5", values="projection_weight", aggfunc='sum').astype(int)
l5 = l5.drop([98, 99], axis=1)

In [385]:
import plotly.graph_objects as go

countries = ["United Kingdom", "Ireland", "France", "Germany", "Spain", "Netherlands"]

fig = go.Figure()
buttons = []
i = 0

for country in countries:
    current = l5.loc[country]
    current = current.rename({1: "Very serious threat", 2: "Somewhat serious threat", 3: "Not a threat at all"}, axis=1)

    for col in current:
        fig.add_trace(
            go.Bar(
                x=current.index.to_list(),
                y=current[col].to_list(),
                name=col,
                visible=(i == 0)
            )
        )

    args = [False] * len(countries) * len(current.columns)
    args[i*len(current.columns):(i*len(current.columns))+len(current.columns)] = [True] * len(current.columns)
    
    button = dict(label = country,
                  method = "update",
                  args=[{"visible": args}])
    
    buttons.append(button)
    
    i += 1

fig.update_layout(
    updatemenus=[
        dict(
            type="dropdown",
            direction="down",
            x=1,
            y=1,
            buttons=buttons
        )
    ],
    barmode="stack"
)

# Set title
fig.update_layout(title_text=relevant_keys.get("L5"))

fig.show()

In [322]:
l5_total = l5.copy()
l5_total = l5_total.groupby("Country").sum()
l5_total["total"] = l5_total[1]/l5_total.sum(axis=1)

In [384]:
import pycountry

input_countries = l5_total.index.to_list()
countries = {country.name : country.alpha_2 for country in pycountry.countries}
codes = [countries.get(country) for country in input_countries]